In [1]:
#Generate text character by character, by looking at Shakespearean

#Import modules for preprocessing
import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

#Text that our model will be looking at
file = unidecode.unidecode(open('./data/shakesphere/text.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1225672


In [2]:
#Picks a chunk of the text
chunk_len = 200

def random_chunk():
    start = random.randint(0, file_len - chunk_len)
    return file[start: start + chunk_len]

In [3]:
#Here's random_chunk in practice
random_chunk()

's in the wreck of maidenhood, cannot for all that\n    dissuade succession, but that they are limed with the twigs that\n    threatens them. I hope I need not to advise you further; but I\n    hope your '

In [4]:
#Importing torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [5]:
#Heres out RNN model. We'll be using an Embedding for encoding, Linear unit unit for decoding and a GRU in between
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out = self.encoder(input.view(1, -1))
        out, hidden = self.gru(out.view(1, 1, -1), hidden)
        out = self.decoder(out.view(1, -1))
        return out, hidden
        
    def initHidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

In [6]:
#Converts text to a tensor of numbers
def charTensor(inp_str):
    out = [all_characters.index(c) for c in inp_str]
    return torch.tensor(out).long()

In [7]:
#charTensor in practice
charTensor("hello")

tensor([ 17,  14,  21,  21,  24])

In [8]:
#Picks a random chunk of text and converts into in a tensor
def randomTrainSample():
    chunk = charTensor(random_chunk())
    inp = chunk[:-1]
    target = chunk[1:]
    return inp, target

In [9]:
randomTrainSample()

(tensor([ 18,  15,  94,  44,  94,  32,  24,  30,  21,  13,  96,  94,
          94,  94,  94,  54,  25,  14,  10,  20,  94,  29,  17,  10,
          29,  74,  96,  94,  94,  54,  44,  38,  44,  49,  44,  56,
          54,  75,  94,  58,  14,  94,  20,  23,  24,  32,  94,  34,
          24,  30,  27,  94,  13,  27,  18,  15,  29,  75,  94,  54,
          25,  14,  10,  20,  94,  32,  17,  10,  29,  82,  96,  94,
          94,  37,  53,  56,  55,  56,  54,  75,  94,  55,  17,  14,
          27,  14,  68,  28,  94,  23,  24,  94,  22,  24,  27,  14,
          94,  29,  24,  94,  11,  14,  94,  28,  10,  18,  13,  73,
          94,  11,  30,  29,  94,  17,  14,  94,  18,  28,  94,  11,
          10,  23,  18,  28,  17,  68,  13,  73,  96,  94,  94,  94,
          94,  36,  28,  94,  14,  23,  14,  22,  34,  94,  29,  24,
          94,  29,  17,  14,  94,  25,  14,  24,  25,  21,  14,  94,
          10,  23,  13,  94,  17,  18,  28,  94,  12,  24,  30,  23,
          29,  27,  34,  75,  96, 

In [10]:
#Training step
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.initHidden()
    prime_input = charTensor(prime_str)
    predicted = prime_str
    
    for p in range(len(prime_str)):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = charTensor(predicted_char)
    
    return predicted

In [11]:
#Trains the model
def train(inp, target):
    hidden = rnn.initHidden()
    rnn.zero_grad()
    
    loss = 0
    for c in range(chunk_len - 1):
        output, hidden = rnn(inp[c], hidden)
        loss += criterion(output, torch.tensor([target[c]]).long())
        
    loss.backward()
    optimizer.step()
    
    return loss.data[0]/chunk_len

In [12]:
#Hyper-parameters
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
input_size = output_size = n_characters
n_layers = 1
lr = 0.005

rnn = RNN(input_size, hidden_size, output_size, n_layers)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [13]:
#Train the model
loss_avg = 0
all_losses = []
for epoch in range(n_epochs):
    loss = train(*randomTrainSample())
    loss_avg += loss
    
    if(epoch%print_every == 0):
        print("Epoch", epoch)
        print(evaluate('Wh', predict_len=100) + '\n')
   
    if(epoch%plot_every == 0):
        all_losses.append(loss)
        loss_avg = 0

D:\ProgramFiles\Anaconda\lib\site-packages\ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


Epoch 0
S[%"^"G_]bhRPkO\R9zrG~"_YMc/u|fQV)M~7]N%R"Z

Epoch 100
Wh[ sore fot pigheed ns thell ors me and spars sen ges oft hinme stset bend that bef ang, wopre am fow

Epoch 200
Wher, to rownd poring cotrer, wot il yuth thats cadis to whe ar ou and wour theur oh tharn thee we ver

Epoch 300
What sore did?
  TMARVDSe. If be hourip bot be my frack my grivy sep be dniser' son bod and moress
   

Epoch 400
Whill
               Weethe?                                                                          

Epoch 500
Whe siser. Dring.
   Noin the saof the she a ellove my so ands, ming and strrais a bus poresestless de

Epoch 600
Whav'd mear'd I hey
        in the reave a as namat aceintts the causing that ment trom a gorn
    Thy

Epoch 700
Who his breast, pears beabt in man falll makinges; cang taken maket grame.
  Hargel sall her nok sario

Epoch 800
Whil, [and theare in EPERSAY FERMIPHOLUS MEOND SAYYRALA, I never our hise is not on CORIOLANT OUTHESTR

Epoch 900
Whert truse woudds not th